### Imports

In [12]:
import numpy as np

import sys

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
from matplotlib.ticker import ScalarFormatter

from astropy import units as u
from astropy import constants as const
from astropy.table import QTable

from pathlib import Path
import os

sys.path.insert(0, '/home/emelie/Desktop/Codes/PETAR/PeTar-master/tools/')
from analysis import *



import import_ipynb
from Visualization_functions import snapshots_plot, anim_disc, anim_3d, anim_ComparetoTraceback

importing Jupyter notebook from Visualization_functions.ipynb


In [3]:
plt.rcParams.update({'xtick.labelsize':13, 'ytick.labelsize':13, 'axes.titlesize':16, 
                     'axes.grid':True, 'axes.labelsize':15, 'legend.fontsize':13})

# Functions

### Extracting data function

In [4]:
def extract_data(name_run, name_folder, tmax, toutput):
    """
    Extracts data from petar snapshot files and checks the data
    ------------------------------------------------------------
    Parameters:
    -----------
    name_run: str
        Name of the run. NOTE! Do not forget to add an r before the string!!!
        
    name_folder: str
        Name of the folder in which the data is kept. NOTE! Do not forget to add an r before the string!!!
        
    tmax: int
        Number of timesteps
    
    toutput: int
        Which timesteps to give snapshots
        
        
    Output:
    --------
    all_data: array
        An array with all data for all particles and all snapshots. The order of the columns is 
        [mass, x, y, z, r, vx, vy, vz, v, r_search, mass_bk, status, acc_sofs x, acc_soft y, acc_soft z, 
        pot_tot, pot_soft, pot_ext]. Positions are in kpc and velocities in km/s
        
    header_values: array
        An array with all data from the headers of the snapshots. The order of the columns is 
        [number of particles, time, x_offset, y_offset, z_offset, vx_offset, vy_offset, vz_offset]. 
        Positions are in kpc and velocities in km/s
    
    extra_data: array
        Calculated extra data that is relevant for the analysis:
        [half_mass_radius, r_mean_particles, r_mean_cluster, v_mean]
        
    hist_data: array
        Data for making a cumulative mass vs radius plot 
        
    """
    n_files = int((tmax/toutput) + 1)
    
    
    # Fixing basic path
    notebook_path = Path.cwd()

    data_path = Path('Result_files')

    total_path = os.path.join(notebook_path, data_path)
    
    # Importing header
    header_values = np.empty((8, n_files)) # (quantities, timesteps)
    extra_data = np.zeros((4, n_files)) # (quantities, timesteps)

    for i in range(n_files):
        header = PeTarDataHeader(total_path+f'/{name_folder}/{name_run}.{i}', 
                                       external_mode='galpy', snapshot_format='ascii')
        header_values[0, i] = header.n # Number of particles
        header_values[1, i] = header.time # Time
        
        x_mean = header.pos_offset[0] # Cluster mean x position
        y_mean = header.pos_offset[1] # Cluster mean y position
        z_mean = header.pos_offset[2] # Cluster mean z position
        header_values[2, i] = x_mean/1000 # Convert to kpc 
        header_values[3, i] = y_mean/1000 # Convert to kpc
        header_values[4, i] = z_mean/1000 # Convert to kpc
        
        r_mean_cluster = np.sqrt(x_mean**2 + y_mean**2 + z_mean**2)
        
        extra_data[2, i]
        
        # Convert from pm/Myr to km/s
        vx_mean = (header.vel_offset[0] * u.pc/u.Myr).to(u.km/u.s).value # Cluster mean velocity in x position
        vy_mean = (header.vel_offset[1] * u.pc/u.Myr).to(u.km/u.s).value # Cluster mean velocity in y position
        vz_mean = (header.vel_offset[2] * u.pc/u.Myr).to(u.km/u.s).value # Cluster mean velocity in z position
        header_values[5, i] = vx_mean 
        header_values[6, i] = vy_mean 
        header_values[7, i] = vz_mean
        
        v_mean_cluster = np.sqrt(vx_mean**2 + vy_mean**2 + vz_mean**2)
        extra_data[3, i] = v_mean_cluster
        
        
        
        
    # Importing data into 3D array
    all_data = np.empty((int(header_values[0, 0]), 21, n_files)) # (particles, quantities, timesteps)
    n_particles = np.empty((n_files))
    hist_data = np.zeros((2, int(header_values[0, 0]), n_files))
    
    for i in range(n_files):
        # Importing the data
        particles = Particle(external_mode='galpy', interupt_mode='bse')
        particles.loadtxt(total_path+f'/{name_folder}/{name_run}.{i}', skiprows=1)
        all_data[:, 0, i] = particles.mass # mass
        
        # Working with the positions
        x = particles.pos[:, 0]/1000 # Convert to kpc
        y = particles.pos[:, 1]/1000 # Convert to kpc
        z = particles.pos[:, 2]/1000 # Convert to kpc
        
        # Adjusting the positions of the particles to the position of the cluster
        all_data[:, 1, i] = x + header_values[2, i] # Galactic centre in origin frame
        all_data[:, 2, i] = y + header_values[3, i]
        all_data[:, 3, i] = z + header_values[4, i]
        
        r = np.sqrt(x**2 + y**2 + z**2) # Cluster centered around the origin frame
        all_data[:, 4, i] = r
        
        extra_data[1, i] = np.mean(r, axis=0) # r_mean_particles
    
        #extra_data[1, i] = np.mean(x, axis=0) # x_mean
        #extra_data[2, i] = np.mean(y, axis=0) # y_mean
        #extra_data[3, i] = np.mean(z, axis=0) # z_mean
        
        # Working with the velocities and convert from pc/Myr to km/s
        vx = (particles.vel[:, 0] * u.pc/u.Myr).to(u.km/u.s).value # vx
        vy = (particles.vel[:, 1] * u.pc/u.Myr).to(u.km/u.s).value # vy
        vz = (particles.vel[:, 2] * u.pc/u.Myr).to(u.km/u.s).value # vz
        all_data[:, 5, i] = vx
        all_data[:, 6, i] = vy
        all_data[:, 7, i] = vz
        
        #extra_data[5, i] = np.mean(vx, axis=0) # vx_mean
        #extra_data[6, i] = np.mean(vy, axis=0) # vy_mean
        #extra_data[7, i] = np.mean(vz, axis=0) # vz_mean
    
        v = np.sqrt(vx**2 + vy**2 + vz**2)
        all_data[:, 8, i] = v
        
        # The rest of the data
        all_data[:, 9, i] = particles.r_search # neighbour searching radius
        all_data[:, 10, i] = particles.id # particle id
        all_data[:, 11, i] = particles.mass_bk # Artificial particle parameter
        all_data[:, 12, i] = particles.status # Artificial particle parameter
        all_data[:, 13, i] = particles.r_in # inner changeover radius
        all_data[:, 14, i] = particles.r_out # outer changeover radius
        all_data[:, 15, i] = particles.acc_soft[:, 0] # long-range acceleration in x
        all_data[:, 16, i] = particles.acc_soft[:, 1] # long-range acceleration in y
        all_data[:, 17, i] = particles.acc_soft[:, 2] # long-range acceleration in z
        all_data[:, 18, i] = particles.pot # total potential
        all_data[:, 19, i] = particles.pot_soft # long-range(?) potential
        all_data[:, 20, i] = particles.pot_ext # external potential
        
        # Checking number of particles
        n_particles[i] = len(particles.mass)
        
        # Half-mass radius
        extra_data[0, i], hist_data[:, :, i] = halfmass_radius_cumsum(particles.mass, r)
    
    # Checking if the number of particles is the same throughout the simulation
    n_part_same = np.all(n_particles == n_particles[0])
    print(f'Number of particles is conserved: {n_part_same}')
    
    
    
    return all_data, header_values, extra_data, hist_data
        
        

### Calculating means

In [5]:
def means_calc(header_array):
    v_mean = np.sqrt(header_array[-3, :]**2 +header_array[-2, :]**2 +header_array[-1, :]**2)
    
    r_mean = np.sqrt(header_array[2, :]**2 + header_array[3, :]**2 + header_array[4, :]**2)
    
    return v_mean, r_mean

### Half-mass radius function

In [6]:
def halfmass_radius_cumsum(masses, radii):
    new_data_array = np.empty((len(masses), 2)) # Make an empty array
    new_data_array[:, 0] = masses # Fills the first column with masses
    new_data_array[:, 1] = radii # Fills the second columns with radii
    sorted_array = np.array(sorted(new_data_array, key=lambda x:x[1])) # Sort the data according to the radii
    
    cumsum_mass = np.cumsum(sorted_array[:, 0]) # Calculate a cumulative sum of all masses, shape(n_particles)
    
    hist_data = np.vstack((cumsum_mass, sorted_array[:, 1]))
    
    Mtot = np.sum(masses)
    half_mass = 0.5*Mtot
    
    difference = np.abs(cumsum_mass - half_mass)
    closest_mass = np.min(difference)
    
    position = np.where(difference==closest_mass)[0]
    
    r_halfmass = sorted_array[position, 1][0]
    
    return r_halfmass, hist_data

### Interpolation of integrated mean orbit

In [7]:
def easy_interpolate(orbit_data, laps):
    """
    Parameters:
    ------------
    orbit_data: 2d array
        Data with Columns: x, y, z, vx, vy, vz, t, for orbit simulation
        
    laps: int
        Number of times to do this
        
    Output:
    -------
    new_data: 4d array
        Orbital positions with columns: x, y, z, vx, vy, vz, t, distance_in_stream
    """
    for i in range(laps):
        # Roll particles to get x1-x2, x2-x3, etc.
        orbit_data_rolled = np.roll(orbit_data, shift=1, axis=0) 
        
        # Delete first particle because of mismatch: xn-x1
        data = np.delete(orbit_data, 0, axis=0) 
        data_rolled = np.delete(orbit_data_rolled, 0, axis=0)
        #print(data_rolled[:10])
        #print(data[:10])
        
        # Calculating middle point position: x, y, z
        interp_pos = (data_rolled[:, 0:3] + data[:, 0:3])/2 
        # Calculating average velocity between positions dx/dt
        dx = data_rolled[:, 0:3] - data[:, 0:3]
        dt = data_rolled[:, -1] - data[:, -1]
        av_vel = dx/dt[:, np.newaxis]
        # Calculating average time between points
        av_t = (data_rolled[:, -1] + data[:, -1])/2
        
        # Length of data with added points in between
        len_new_data = 2*len(orbit_data)-1
        #print(len_new_data)

        # x, y, z, vx, vy, vz, t
        new_data = np.zeros((len_new_data, 7))
        
        # Adding data for new points
        new_data[0::2, :] = orbit_data
        new_data[1::2, 0:3] = interp_pos
        new_data[1::2, 3:6] = av_vel
        new_data[1::2, -1] = av_t
        
        orbit_data = new_data
    
    new_data_rolled = np.roll(new_data, shift=1, axis=0)
    orb_dat = np.delete(new_data, 0, axis=0)
    orb_dat_rolled = np.delete(new_data_rolled, 0, axis=0)

    # Distance between consecutive points
    dists = np.sqrt(np.sum((orb_dat_rolled[:, 0:3] - orb_dat[:, 0:3])**2, axis=1))
    
    distance_in_stream = np.zeros(len(new_data))
    for i in range(1, len(dists)+1):
        distance_in_stream[i] = np.sum(dists[:i+1])
    
    #print(dists[:10])
    #print(np.shape(new_data), np.shape(dists_stream))
    
    # Gives columns x, y, z, vx, vy, vz, t, dists
    new_data = np.concatenate((new_data, distance_in_stream[:, np.newaxis]), axis=1)
    
    return new_data

### Finding stream crossing

In [8]:
def stream_crossing(orbit_data, dist_upper_lim, vel_lower_lim):
    orbit_positions = orbit_data[2:5, :].T # Shape: (Norb, 3)
    orbit_velocities = orbit_data[5:8, :].T # Shape: (Norb, 3)
    
    # Calculating distances between all points in orbit
    # Shape(Norb, Norb, 3)
    distances_all_pos = orbit_positions[:, np.newaxis, :] - orbit_positions[np.newaxis, :, :] 
    # Shape(Norb, Norb)
    r_diff_all_pos = np.sqrt(np.sum(distances_all_pos**2, axis=2))
    
    # Calculating difference between velocities between all points in orbit
    # Shape(Norb, Norb, 3)
    differences_all_vel = orbit_velocities[:, np.newaxis, :] - orbit_velocities[np.newaxis, :, :]
    # Shape(Norb, Norb)
    v_diff_all_pos = np.sqrt(np.sum(differences_all_vel**2, axis=2))
    
    # Making masks to find stream crossing
    cross_pos_mask = (r_diff_all_pos>0)&(r_diff_all_pos<=dist_upper_lim)
    cross_vel_mask = (v_diff_all_pos>=vel_lower_lim)
    # Combining masks
    crossings_mask = cross_pos_mask*cross_vel_mask
    
    # Finding indexes for the crossing points
    positions = np.where(crossings_mask==True)[0]
    
    # Extracting position data for crossing
    crossing_dat = orbit_positions[positions]
    
    return crossing_dat

### Stream alignment step 1

In [10]:
def stream_alignment_step1(orbit_data, stream_data): # stream_crossing_pnt, crossing_dist_lim
    """
    Parameters:
    -----------
    orbit_data: 2d array
        Data from mean orbit containing x, y, z, vx, vy, vz, inside_dists
        
    stream_data: 2d array
        Data from final stream containing x, y, z, vx, vy, vz
        
    #stream_crossing_pnt: 2d array
    #    Position where the streams cross each other
    
    """
    
    # Calculating distances to orbit
    #------------------------------------------------------------------------------------------------------
    # Creating empty arrays to have data for stream distance and mean orbit distance
    min_dist_to_orb = np.zeros((len(stream_data), 1))
    dist_in_orb = np.zeros((len(stream_data), 1))
    
    # Extracting inside orbit distances from orbit data
    inside_orbit_dists = orbit_data[:, -1]
    
    # Calculating differences between particles positions and all orbit positions (x, y, z)
    # (N_part, 1, 3) - (n_orb, 3), skipping last column hence [:, :-1]
    # Should give shape N_part, n_orb, 3
    differences_to_orbit = stream_data[:, np.newaxis, 0:3] - orbit_data[:, 0:3]
    print('Differences calculated')
    
    # Calculating sqrt(x² + y² + z²), has shape (N_part, n_orb, 1)
    distances_to_orbit = np.sqrt(np.sum(differences_to_orbit**2, axis=2))
    #print(np.shape(differences_to_orbit))
    
    
    # Finding minimum distances to orbit and connecting to distance inside stream
    # --------------------------------------------------------------------------------------------
    # Finding minimum distances, i.e. clostest orbit particle distances
    min_dists = np.min(distances_to_orbit, axis=1)
    #print(np.shape(min_dists))
    
    # Adding minimum distances to new array
    min_dist_to_orb = min_dists
    
    # Finding the positions for the closest particles
    pos_min_dists = np.argmin(distances_to_orbit, axis=1)
    #print(np.shape(pos_min_dists))
    
    # Extracting inside stream distances from orbit data points
    stream_distances = orbit_data[pos_min_dists, 6]
    
    # Adding inside stream distances second new array
    dist_in_orb = stream_distances
    
    # Adding new columns to stream data
    # Has shape x, y, z, vx, vy, vz, dist_closest_orb_part, dist_in_stream
    new_stream_data = np.concatenate([stream_data, min_dist_to_orb[:, np.newaxis], dist_in_orb[:, np.newaxis]], 
                                     axis=1) 
        
    return new_stream_data

### Stream alignment step 2

In [29]:
def stream_alignment_step2(orbit_data, new_stream_data):
    
    mean_dist_column = np.zeros((len(new_stream_data), 1))
    # Has shape x, y, z, vx, vy, vz, distance to closest orbit particle, distance in stream, 
    # mean distance to mean orbit
    aligned_stream_data = np.concatenate([new_stream_data, mean_dist_column], axis=1)

    for i, st_dist in enumerate(orbit_data[:, -1]):
        # Mask finding all particles at a certain distance inside the stream
        particles_mask = (new_stream_data[:, -1]==st_dist)
        # Extracting their minimum distances to the orbit
        particles_orb_dists = new_stream_data[particles_mask, -2]
        #print(len(particles_orb_dists))
        # Calculating the mean in this position in the stream
        mean_dist = np.mean(particles_orb_dists)
        # Correcting the distance to the orbit with mean distance
        aligned_stream_data[particles_mask, -1] = aligned_stream_data[particles_mask, 6] - mean_dist

    
    
    return aligned_stream_data

### Extracting stream shape properties function

In [41]:
def stream_shape(orbit_data, aligned_stream_data):
    
    length = np.max(aligned_stream_data[:, 7]) - np.min(aligned_stream_data[:, 7])
    width = np.zeros((len(orbit_data), 1))
    part_dens = np.zeros((len(orbit_data), 1))
    
    # Calculating width and particle density along the stream
    for i, st_dist in enumerate(orbit_data[:, -1]):
        # Mask finding all particles at a certain distance inside the stream
        particles_mask = (aligned_stream_data[:, -2]==st_dist)
        
        # Extracting their corrected minimum distances to the orbit
        particles_corr_orb_dists = aligned_stream_data[particles_mask, -1]
        #print(len(particles_corr_orb_dists))
        
        if len(particles_corr_orb_dists)==0:
            part_dens[i] = 0
            width[i] = 0
            
        else:
            part_dens[i] = len(particles_corr_orb_dists)
            width[i] = np.max(particles_corr_orb_dists) - np.min(particles_corr_orb_dists)
        
        
    return length, width, part_dens

### Calculate relative difference using bins function

In [ ]:
def binned_relative_difference(stream_prop, stream_data, comp_prop, comp_data, orbit_data, n_bins):
    min_stream_length = np.min(orbit_data[:, -1])
    max_stream_length = np.max(orbit_data[:, -1])
    
    # arange for setting bin width instead of number of bins
    bin_edges = np.linspace(min_stream_length, max_stream_length, n_bins)
    
    bin_edges_rolled = np.roll(bin_edges, 1)
    bin_edges_rolled_corr = np.delete(bin_edges_rolled, 0)
    bin_edges_corr = np.delete(bin_edges, 0)
    bin_positions = (bin_edges_rolled_corr + bin_edges_corr)/2
    
    relative_difference = []
    standard_dev = []
    
    for i in range(1, len(bin_edges)):
        #bin_mask_stream = (bin_edges[i-1]<=stream_data[:, -2])&(stream_data[:, -2]<bin_edges[i])
        bin_mask_comp = (bin_edges[i-1]<=orbit_data[:, -1])&(orbit_data[:, -1]<bin_edges[i])
        
        property_stream = stream_prop[bin_mask_comp]
        property_comp = comp_prop[bin_mask_comp]
        
        mean_prop_stream = np.median(property_stream)
        mean_prop_comp = np.median(property_comp)
        
        rel_diff_mean = np.abs((mean_prop_comp - mean_prop_stream)/(mean_prop_comp))
        relative_difference.append(rel_diff_mean)
        
        # Calculating standard deviation
        stream_std = np.std(property_stream)
        comp_std = np.std(property_comp)
        
        std_den = comp_std + stream_std
        std_frac = rel_diff_mean*(std_den/(mean_prop_comp - mean_prop_stream) + comp_std/mean_prop_comp)
        standard_dev.append(std_frac)
        
        
    
    relative_difference = np.array(relative_difference)
    relative_difference[np.isnan(relative_difference)] = 0
    relative_difference[np.isinf(relative_difference)] = np.max(relative_difference)
    
    standard_dev = np.array(standard_dev)
    
    result = np.concatenate([bin_positions[:, np.newaxis], relative_difference[:, np.newaxis], 
                             standard_dev[:, np.newaxis]], axis=1)
    
    return result #bin_positions, relative_difference

### Inertia tensor fcns

The intertia tensor for N particles is given by:


\begin{equation}
I_{tensor} = 
\begin{pmatrix} 
\sum\limits_{i=1}^N m_i (y_i^2 + z_i²) & -\sum\limits_{i=1}^N m_i x_i y_i & -\sum\limits_{i=1}^N m_i x_i z_i \\ 
-\sum\limits_{i=1}^N m_i x_i y_i & \sum\limits_{i=1}^N m_i (x_i^2 + z_i²) & -\sum\limits_{i=1}^N m_i y_i z_i \\
-\sum\limits_{i=1}^N m_i x_i z_i & -\sum\limits_{i=1}^N m_i y_i z_i & \sum\limits_{i=1}^N m_i (x_i^2 + y_i²) \\
\end{pmatrix} = 
\begin{pmatrix} 
I_{xx} & -I_{xy} & -I_{xz} \\ 
-I_{yx} & I_{yy} & -I_{yz} \\
-I_{zx} & -I_{zy} & I_{zz} \\
\end{pmatrix}
\end{equation}

In [ ]:
def Iii(masses, j, k):
    """
    masses: array
        Contains masses of all particles. Shape: (N)
        
    j: array
        Contains positions in j direction. E.g. if main axis is x, j=y and k=z
    """
    coord = j**2 + k**2
    tot = np.sum(masses*coord, axis=0)
    return tot


def Iij(masses, i, j, k):
    coord1 = masses*i*j 
    coord2 = masses*i*k 
    
    tot1 = np.sum(coord1, axis=0)
    tot2 = np.sum(coord2, axis=0)
    
    return tot1, tot2


def inertia_tensor(masses, pos):
    """
    pos: Shape (N, 3), where 3 => x, y, z
    """
    tensor = np.zeros((3, 3))
    
    for i in range(3):
        coord = np.array([0, 1, 2])
        indexes = np.delete(coord, i)
        
        tensor[i, i] = Iii(masses, pos[:, indexes[0]], pos[:, indexes[1]])
        tens1, tens2 = Iij(masses, pos[:, i], pos[:, indexes[0]], pos[:, indexes[1]])
        tensor[i, indexes[0]] = -tens1
        tensor[i, indexes[1]] = -tens2
    
    
    return tensor

### Calculating aspect ratios from inertia tensor eigenvalues function

From equation (2) in Robles et al. 2015

\begin{equation}
    a = \sqrt{\frac{5 (\lambda_2 - \lambda_1 + \lambda_3)}{2M_{tot}}}
\end{equation}

\begin{equation}
    b = \sqrt{\frac{5 (\lambda_3 - \lambda_2 + \lambda_1)}{2M_{tot}}}
\end{equation}


\begin{equation}
    c = \sqrt{\frac{5 (\lambda_1 - \lambda_3 + \lambda_2)}{2M_{tot}}}
\end{equation}

In [ ]:
def aspect_ratios(i1, i2, i3, Mtot):

    a_num = 5*(i2 - i1 + i3)
    b_num = 5*(i3 - i2 + i1)
    c_num = 5*(i1 - i3 + i2)

    den = 2*Mtot

    a = np.sqrt(a_num/den)
    b = np.sqrt(b_num/den)
    c = np.sqrt(c_num/den)
    
    return a, b, c

# Tests

## Fiducial model run, large: 997967particles, 5Gyr forward

Leo T-like progenitor in Boo III's orbit. Using data without uncertainties:

- $M_{dyn} = 9 \,$M$_{\odot}$
- $r_h = 153 \,$pc
- Maschberger 2012 IMF with masses above $2.955 \,$M$_{\odot}$

%%time
run_fid_large_mod_data, run_fid_large_mod_header, run_fid_large_mod_extra, fid_large_mod_hist_data = extract_data(r'fiducial_large_run_restart', 
                                                                                          r'Fiducial_large_run', 39, 1)

print(run_fid_large_mod_data.shape)
print(run_fid_large_mod_extra.shape)
print(run_fid_large_mod_header.shape)
print(run_fid_large_mod_header[:, -1])

fid_large_mod_halfmass_r = run_fid_large_mod_extra[0, :]
print(fid_large_mod_halfmass_r)

print(np.max(fid_large_mod_halfmass_r))
print(np.min(fid_large_mod_halfmass_r))

v_mean_fid_large_mod, r_mean_fid_large_mod = means_calc(run_fid_large_mod_header)

print(v_mean_fid_large_mod.shape)
print(r_mean_fid_large_mod.shape)

In [38]:
#snapshots_btest = [0, 14, 27, -1]
#snapshots_plot(run_fid_large_mod_data, snapshots_btest, 25.0e4, 25.0e4, (11, 20), 1, 'Fiducial_large_model', 
#               save=False)

## Trace forward of Boo III Orbit, 3 particles, 7Gyr forward in time

run_petar_trace_forward_data, run_petar_trace_forward_header, run_petar_trace_forward_extra, petar_trace_forward_hist_data = extract_data(r'Trace_forward_run4_data', r'Trace_forward_run4', 1750, 1)

print(run_petar_trace_forward_data.shape)
print(run_petar_trace_forward_extra.shape)
print(run_petar_trace_forward_header.shape)

petar_trace_forward_halfmass_r = run_petar_trace_forward_extra[0, :]
#print(petar_trace_forward_halfmass_r)

print(np.max(petar_trace_forward_halfmass_r))
print(np.min(petar_trace_forward_halfmass_r))

v_mean_petar_trace_forward, r_mean_petar_trace_forward = means_calc(run_petar_trace_forward_header)

print(v_mean_petar_trace_forward.shape)
print(r_mean_petar_trace_forward.shape)

### Plotting orbit

fig_orb= plt.figure(figsize=(10, 8))
ax_orb = fig_orb.add_subplot(projection='3d', computed_zorder=False)

u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)


x_bulge = 1500 * np.outer(np.cos(u), np.sin(v))
y_bulge = 1500 * np.outer(np.sin(u), np.sin(v))
z_bulge = 1500 * np.outer(np.ones(np.size(u)), np.cos(v))

x_disc = 14490 * np.outer(np.cos(u), np.sin(v))
y_disc = 14490 * np.outer(np.sin(u), np.sin(v))
z_disc = 300 * np.outer(np.ones(np.size(u)), np.cos(v))
#Before: 40:109
ax_orb.plot(run_petar_trace_forward_header[2, 650:], run_petar_trace_forward_header[3, 650:], 
            run_petar_trace_forward_header[4, 650:], color='b')

sc = ax_orb.scatter(run_petar_trace_forward_header[2, 650:], run_petar_trace_forward_header[3, 650:], 
            run_petar_trace_forward_header[4, 650:], c=run_petar_trace_forward_header[1, 650:], cmap='spring')

plt.colorbar(sc)
ax_orb.set_xlabel('x [pc]', labelpad=15)
ax_orb.set_ylabel('y [pc]', labelpad=15)
ax_orb.set_zlabel('z [pc]', labelpad=5)
ax_orb.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_orb.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax_orb.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_orb.yaxis.get_major_formatter().set_powerlimits((0, 0))
ax_orb.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_orb.zaxis.get_major_formatter().set_powerlimits((0, 0))

ax_orb.plot_surface(x_bulge, y_bulge, z_bulge, color='grey', alpha=0.5)
ax_orb.plot_surface(x_disc, y_disc, z_disc, color='grey', alpha=0.5)

ax_orb.view_init(4, 15)

plt.show()

### Finding crossing

orbit_crossing = stream_crossing(run_petar_trace_forward_header[:, 650:], 5e2, 1e2)
print(orbit_crossing)

orbit_pos = run_petar_trace_forward_header[2:5, 650:].T

difference_to_crossing = orbit_pos - orbit_crossing[0, :]
distance_to_crossing = np.sqrt(np.sum(difference_to_crossing**2, axis=1))

close_orbit_pos = distance_to_crossing<=5e3
close_pos = orbit_pos[close_orbit_pos]
print(np.shape(close_pos))

### Plotting particles close to crossing

fig_orb2 = plt.figure(figsize=(10, 8))
ax_orb2 = fig_orb2.add_subplot(projection='3d', computed_zorder=False)

u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)


x_bulge = 1500 * np.outer(np.cos(u), np.sin(v))
y_bulge = 1500 * np.outer(np.sin(u), np.sin(v))
z_bulge = 1500 * np.outer(np.ones(np.size(u)), np.cos(v))

x_disc = 14490 * np.outer(np.cos(u), np.sin(v))
y_disc = 14490 * np.outer(np.sin(u), np.sin(v))
z_disc = 300 * np.outer(np.ones(np.size(u)), np.cos(v))
#Before: 40:109
#ax_orb.plot(run_petar_trace_forward_header[2, 650:], run_petar_trace_forward_header[3, 650:], 
#run_petar_trace_forward_header[4, 650:], color='b')

sc = ax_orb2.scatter(run_petar_trace_forward_header[2, 650:], run_petar_trace_forward_header[3, 650:], 
                     run_petar_trace_forward_header[4, 650:], c=run_petar_trace_forward_header[1, 650:], 
                     cmap='spring', s=10)

plt.colorbar(sc)

ax_orb2.scatter(orbit_crossing[0, 0], orbit_crossing[0, 1], orbit_crossing[0, 2], c='b', marker='x', s=20)
ax_orb2.scatter(close_pos[:, 0], close_pos[:, 1], close_pos[:, 2], c='r', s=10)


ax_orb2.set_xlabel('x [pc]', labelpad=15)
ax_orb2.set_ylabel('y [pc]', labelpad=15)
ax_orb2.set_zlabel('z [pc]', labelpad=5)
ax_orb2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_orb2.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax_orb2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_orb2.yaxis.get_major_formatter().set_powerlimits((0, 0))
ax_orb2.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_orb2.zaxis.get_major_formatter().set_powerlimits((0, 0))

ax_orb2.plot_surface(x_bulge, y_bulge, z_bulge, color='grey', alpha=0.5)
ax_orb2.plot_surface(x_disc, y_disc, z_disc, color='grey', alpha=0.5)

#ax_orb2.set_xlim(xmin=-0.0e5, xmax=0.4e5)
#ax_orb2.set_ylim(ymin=-0.1e5, ymax=0.0e5)
#ax_orb2.set_zlim(zmin=-0.05e5, zmax=0.05e5)

ax_orb2.view_init(4, 15)

plt.show()

### Constructing orbit data array

orbit_position = run_petar_trace_forward_header[2:5, 650:].T # To get shape (N,x)
orbit_velocity = run_petar_trace_forward_header[5:8, 650:].T # To get shape (N,x)
orbit_times = run_petar_trace_forward_header[1, 650:].T # To get shape (N,x)

#gets columns (x, y, z, vx, vy, vz, t)
orbit_data = np.concatenate([orbit_position, orbit_velocity, orbit_times[:, np.newaxis]], axis=1)

print(np.shape(orbit_data)) # Expect shape n,7

### Interpolating orbit

interpolated_orbit = easy_interpolate(orbit_data, 2)
print(np.shape(interpolated_orbit))
#print(interpolated_orbit[:20, 3:6])

### Testing stream alignment

#Has columns x, y, z, vx, vy, vz
stream_data = run_fid_large_mod_data[:, 1:7, -1]
orbital_data = np.delete(interpolated_orbit, 6, axis=1) # Removing time from orbital data as it is not needed
print(np.shape(orbital_data))

aligned_stream1 = stream_alignment_step1(orbital_data, stream_data)

## Developing alignment function

min_dist_to_orb = np.zeros((len(stream_data), 1))
dist_in_orb = np.zeros((len(stream_data), 1))
    
#Extracting inside orbit distances from orbit data
inside_orbit_dists = orbit_data[:, -1]
    
#Calculating differences between particles positions and all orbit positions (x, y, z)
#(N_part, 1, 3) - (n_orb, 3), skipping last column hence [:, :-1]
#Should give shape N_part, n_orb, 3
differences_to_orbit = stream_data[:, np.newaxis, 0:3] - orbit_data[:, 0:3]
print('Differences calculated')
    
#Calculating sqrt(x² + y² + z²), has shape (N_part, n_orb, 1)
distances_to_orbit = np.sqrt(np.sum(differences_to_orbit**2, axis=2))
#print(np.shape(differences_to_orbit))

### Finding particles close to crossing point

####Doing separate calculations for crossing particles
#Calculating distance to stream crossing point
difference_to_crossing = positions - orbit_crossing[0, :]
distance_to_crossing = np.sqrt(np.sum(difference_to_crossing**2, axis=1))
print(np.min(distance_to_crossing))

#Making a mask to find particles closest to the crossing point
close_orbit_pos = (distance_to_crossing<=5e3) # 5e3
#Extracting particles that are close to crossing, Shape: (N_cross_part, 6)
close_crossing_particles_pos = positions[close_orbit_pos, :]
print(len(close_crossing_particles_pos))

#Extracting velocities from stream data
velocities = run_fid_large_mod_data[:, 4:7, -1]
#Finding velocity data for particles close to crossing point
close_crossing_particles_vel = velocities[close_orbit_pos, :]


#Which rows to find the close to crossing particles
#distances_to_orbit[close_pos_rows, :] gives me the values for close_crossing_particles
close_pos_rows = np.where(close_orbit_pos==True)[0] #, close_pos_cols
#print(close_pos_rows)
print(np.shape(close_pos_rows))

### Finding wrong velocity directions

def scalar_product(a, b):
    # a shape: (N_part, 3)
    # b shape: (N_orb, 3)
        
    if len(a)==1:
        x = a[:, 0]*b[:, 0]
        y = a[:, 1]*b[:, 1]
        z = a[:, 2]*b[:, 2]
        all_pos = np.concatenate([x[:, np.newaxis], y[:, np.newaxis], z[:, np.newaxis]], axis=1)
        summation = x+y+z #np.sum(all_pos, axis=1)
        
    else:
    
        multiplication = a[:, np.newaxis, :]*b[:, :]
        # Shapes (N_part, N_orb, 1)
        x = a[:, np.newaxis, 0]*b[:, 0]
        y = a[:, np.newaxis, 1]*b[:, 1]
        z = a[:, np.newaxis, 2]*b[:, 2]

        all_pos = np.concatenate([x[:, :, np.newaxis], y[:, :, np.newaxis], z[:, :, np.newaxis]], axis=2)

        summation = np.sum(all_pos, axis=2) #multiplication
    return summation

#Checking velocity directions through division, vx, vy and vz directions positive=> same direction
#Should give shape N_close_part, n_orb, 3
velocity_directions = close_crossing_particles_vel[:, np.newaxis, :]/interpolated_orbit[:, 3:6]
#print(velocity_directions)
#print('Velocity directions calculated')

#Gives me shape N_cross_part, n_orb
wrong_direction_mask = np.any(velocity_directions<0, axis=2)
#print(wrong_direction_mask[:10, :])

#right_direction_mask = np.all(velocity_directions>0, axis=2)
#print(velocity_directions[right_direction_mask])
#print(len(velocity_directions[right_direction_mask]))

scalar_prod = scalar_product(close_crossing_particles_vel, interpolated_orbit[:, 3:6])
print(f'Shape scalar product {np.shape(scalar_prod)}')
stream_vector_lengths = np.sqrt(np.sum(close_crossing_particles_vel**2, axis=1))
orbit_vector_lengths = np.sqrt(np.sum(interpolated_orbit[:, 3:6]**2, axis=1))
print(f'Shape stream vector lengths {np.shape(stream_vector_lengths)}')
print(f'Shape orbit vector lengths {np.shape(orbit_vector_lengths)}')
print(f'Orbit vector lenghts {orbit_vector_lengths}')

one_part = close_crossing_particles_vel[0, :]
print(np.shape(one_part))
scalar = scalar_product(one_part[np.newaxis, :], interpolated_orbit[:, 3:6])
#scalar = np.dot(one_part[np.newaxis, :], interpolated_orbit[:, 3:6])
print(scalar)
length_one_part = np.sqrt(np.sum(one_part))
print(length_one_part)
div = scalar/(length_one_part*orbit_vector_lengths)
print(length_one_part*orbit_vector_lengths)
print(div)
angle = np.arccos(div)
print(angle)

#In radians
angles = np.arccos(scalar_prod/(stream_vector_lengths[:, np.newaxis]*orbit_vector_lengths[np.newaxis, :]))
print(np.shape(angles))
print(f'{np.min(angles) * 180/np.pi} deg', np.min(angles))
print(f'{np.max(angles) * 180/np.pi} deg', np.max(angles))
#print(angles* 180/np.pi)

#print(80*np.pi/180)

min_ang_pos = np.argmin(angles, axis=1)
print(np.shape(min_ang_pos))
#wrong_direction_mask =  angles>1.34
#Makes distances with wrong directions super large
#distaces_to_orbit[close_pos_rows, :][wrong_direction_mask] = 1e20


distances_to_orbit[close_pos_rows, :][:, min_ang_pos] = distances_to_orbit[close_pos_rows, :][:, min_ang_pos]/100
####
#print(np.shape(distances_to_orbit[close_pos_rows, :][wrong_direction_mask]))

### Finding minimum distances to orbit

#Finding minimum distances to orbit and connecting to distance inside stream
#--------------------------------------------------------------------------------------------
#Finding minimum distances, i.e. clostest orbit particle distances
min_dists = np.min(distances_to_orbit, axis=1)
#print(np.shape(min_dists))
    
#Adding minimum distances to new array
min_dist_to_orb = min_dists
    
#Finding the positions for the closest particles
pos_min_dists = np.argmin(distances_to_orbit, axis=1)
print(np.shape(pos_min_dists))
    
#Extracting inside stream distances from orbit data points
stream_distances = orbit_data[pos_min_dists, 6]
    
#Adding inside stream distances second new array
dist_in_orb = stream_distances
    
#Adding new columns to stream data
#Has shape x, y, z, vx, vy, vz, dist_closest_orb_part, dist_in_stream
new_stream_data = np.concatenate([stream_data, min_dist_to_orb[:, np.newaxis], dist_in_orb[:, np.newaxis]], 
                                     axis=1)

### Plotting aligned stream part 1

fig_t1, ax_t1 = plt.subplots(figsize=(12, 6))

ax_t1.scatter(aligned_stream1[:, -1], aligned_stream1[:, -2], c='b', s=5)
#ax_t1.scatter(aligned_stream_st1[:, 7], aligned_stream_st1[:, 6], c='b', s=5)

ax_t1.set_title('Stream aligned with mean orbit')
ax_t1.set_xlabel('Distance inside stream [pc]')
ax_t1.set_ylabel('Distance to mean orbit [pc]')

ax_t1.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t1.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax_t1.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t1.yaxis.get_major_formatter().set_powerlimits((0, 0))


plt.show()

### Checking if the crossing is working

fig_t2 = plt.figure(figsize=(10, 8))
ax_t2 = fig_t2.add_subplot(projection='3d', computed_zorder=False)

u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)


x_bulge = 1500 * np.outer(np.cos(u), np.sin(v))
y_bulge = 1500 * np.outer(np.sin(u), np.sin(v))
z_bulge = 1500 * np.outer(np.ones(np.size(u)), np.cos(v))

x_disc = 14490 * np.outer(np.cos(u), np.sin(v))
y_disc = 14490 * np.outer(np.sin(u), np.sin(v))
z_disc = 300 * np.outer(np.ones(np.size(u)), np.cos(v))
    


#Plotting stream coloured with stream distance
sc = ax_t2.scatter(run_fid_large_mod_data[:, 1, -1], run_fid_large_mod_data[:, 2, -1], 
                     run_fid_large_mod_data[:, 3, -1], c=aligned_stream1[:, -1], cmap='spring', s=10, zorder=0,
                     label='Stream data') #alpha=0.5, 


cbar = plt.colorbar(sc, label='Stream distances [pc]', format=ScalarFormatter(useMathText=True))
cbar.formatter.set_powerlimits((0, 0))

#ax_orb3.scatter(close_crossing_particles_pos[:, 0], close_crossing_particles_pos[:, 1],
#close_crossing_particles_pos[:, 2], c='r', label='Particles close to crossing')



ax_t2.set_xlabel('x [pc]', labelpad=15)
ax_t2.set_ylabel('y [pc]', labelpad=15)
ax_t2.set_zlabel('z [pc]', labelpad=5)
ax_t2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t2.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax_t2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t2.yaxis.get_major_formatter().set_powerlimits((0, 0))
ax_t2.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t2.zaxis.get_major_formatter().set_powerlimits((0, 0))

ax_t2.plot_surface(x_bulge, y_bulge, z_bulge, color='grey', alpha=0.5)
ax_t2.plot_surface(x_disc, y_disc, z_disc, color='grey', alpha=0.5)

ax_t2.set_xlim(xmin=-0.0e5, xmax=0.4e5)
ax_t2.set_ylim(ymin=-0.05e5, ymax=0.05e5)
ax_t2.set_zlim(zmin=-0.05e5, zmax=0.05e5)

ax_t2.view_init(4, 55)

plt.show()

### Corrected alignment

aligned_stream2 = stream_alignment_step2(orbital_data, aligned_stream1)

fig_t3, ax_t3 = plt.subplots(figsize=(12, 6))

ax_t3.scatter(aligned_stream2[:, -2], aligned_stream2[:, -1], c='b', s=5)

ax_t3.set_title('Corrected alignment')
ax_t3.set_xlabel('Distance inside stream [pc]')
ax_t3.set_ylabel('Distance to mean orbit [pc]')

ax_t3.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t3.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax_t3.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t3.yaxis.get_major_formatter().set_powerlimits((0, 0))


plt.show()

### Testing extracting shape

length, width, part_dens = stream_shape(orbital_data, aligned_stream2)

print(length)
#print(width)
#print(part_dens)

fig_t4, ax_t4 = plt.subplots(1, 2, figsize=(15, 6))

ax_t4[0].scatter(orbital_data[:, -1], width, c='b', s=10)

ax_t4[0].set_title('Width along stream')
ax_t4[0].set_xlabel('Distance inside stream [pc]')
ax_t4[0].set_ylabel('Stream width [pc]')
ax_t4[0].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t4[0].xaxis.get_major_formatter().set_powerlimits((0, 0))
ax_t4[0].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t4[0].yaxis.get_major_formatter().set_powerlimits((0, 0))


ax_t4[1].scatter(orbital_data[:, -1], part_dens, c='b', s=10)

ax_t4[1].set_title('Particle density along stream')
ax_t4[1].set_xlabel('Distance inside stream [pc]')
ax_t4[1].set_ylabel('Particle density')
ax_t4[1].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t4[1].xaxis.get_major_formatter().set_powerlimits((0, 0))
ax_t4[1].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t4[1].yaxis.get_major_formatter().set_powerlimits((0, 0))


plt.show()

fig_t5, ax_t5 = plt.subplots(figsize=(15, 6))

ax_t5.scatter(orbital_data[:, -1], part_dens, c='b', s=10)

ax_t5.set_title('Particle density along stream')
ax_t5.set_xlabel('Distance inside stream [pc]')
ax_t5.set_ylabel('Particle density')
ax_t5.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t5.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax_t5.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax_t5.yaxis.get_major_formatter().set_powerlimits((0, 0))

ax_t5.set_ylim(ymin=0, ymax=0.4e3)

plt.show()

## Angular momentum calculation

\begin{equation}
    \overrightarrow{L} = \overrightarrow{r} \times m\overrightarrow{v}
\end{equation}

positions = run_fid_large_mod_data[:, 1:4, -1] #(Particles, values)
#np.savetxt('Large_fiducial_model_positions.txt', positions, delimiter=',')
#print(np.shape(positions))
velocities = run_fid_large_mod_data[:, 4:7, -1] #(Particles, values)
#print(np.shape(velocities))
masses = run_fid_large_mod_data[:, 0, -1]
#print(np.shape(masses))


mv = masses[:, np.newaxis]*velocities
angular_momenta = np.cross(positions, mv)
ang_mom_tot = np.sqrt(angular_momenta[:, 0]**2 + angular_momenta[:, 1]**2 + angular_momenta[:, 2]**2)

matrix_6d = np.empty([997967, 7])
matrix_6d[:, :3] = positions
matrix_6d[:, 3:6] = angular_momenta
matrix_6d[:, -1] = ang_mom_tot

ang_mom_pos_mask = angular_momenta>0
ang_mom_min_mask = angular_momenta<0

ang_mom_dir = np.empty([997967, 3])
ang_mom_dir[ang_mom_pos_mask] = 1
ang_mom_dir[ang_mom_min_mask] = -1


dist_6d = np.sqrt(positions[:, 0]**2 + positions[:, 1]**2 + positions[:, 2]**2 + velocities[:, 0]**2 + velocities[:, 1]**2 + velocities[:, 2]**2)



print(np.shape(run_fid_large_mod_data[:,1, -1]))
print(np.shape(angular_momenta[:, 0]))

fig, ax = plt.subplots(1, 2,figsize=(14, 6))

ellipse = mpl.patches.Ellipse((0, 0), width=30000, height=300, color='b', alpha=1) # Disc (x, z) view
circle1 = mpl.patches.Circle((0, 0), 1500, color='b', alpha=1) # Bulge (x, z) view

arrow_numbers = 10
colormap=mpl.cm.Reds
color_vals = angular_momenta[::arrow_numbers, 0]
arrow_map = mpl.cm.ScalarMappable(cmap=colormap, norm=None)
arrow_map.set_array(color_vals)
#arrow_map.set_clim(vmin=-2.0e9, vmax=2.0e9)


sc = ax[0].scatter(run_fid_large_mod_data[:,1,-1], run_fid_large_mod_data[:,3,-1], c=dist_6d, alpha=0.3, s=1, cmap='spring')
#arrows0 = ax[0].quiver(run_fid_large_mod_data[::arrow_numbers,1, -1], run_fid_large_mod_data[::arrow_numbers,3, -1], 
#                       ang_mom_dir[::arrow_numbers, 0], ang_mom_dir[::arrow_numbers, 2], scale_units='width', 
#                       scale=50, width=0.0075, color=colormap(color_vals), alpha=1) 

plt.colorbar(sc, ax=ax[0], label='6D distance')
#plt.colorbar(arrow_map, ax=ax[0], label='x-dir')
#ax[0].add_patch(circle1)
#ax[0].add_patch(ellipse)
#ax[0].set_xlim(xmin=-0.7e5, xmax=0.8e5) #xmin=-0.7e5, xmax=0.8e5
#ax[0].set_ylim(ymin=-1.0e5, ymax=1.75e5)
ax[0].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax[0].xaxis.get_major_formatter().set_powerlimits((0, 0))
ax[0].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax[0].yaxis.get_major_formatter().set_powerlimits((0, 0))

#ax[0].set_title('')
ax[0].set_xlabel('x [pc]')
ax[0].set_ylabel('z [pc]')






ax[1].scatter(run_fid_large_mod_data[:,1, -1], run_fid_large_mod_data[:,3, -1], c='b', alpha=0.3, s=1)
arrows1 = ax[1].quiver(run_fid_large_mod_data[::arrow_numbers,1, -1], 
                       run_fid_large_mod_data[::arrow_numbers,3, -1], ang_mom_dir[::arrow_numbers, 0], 
                       ang_mom_dir[::arrow_numbers, 2], scale_units='width', scale=50, 
                       width=0.0075, color=colormap(color_vals), alpha=1) 

plt.colorbar(arrow_map, ax=ax[1], label='x-dir')
ax[1].set_xlim(xmin=0.25e5, xmax=0.5e5)
ax[1].set_ylim(ymin=-0.75e5, ymax=0.0e5)
ax[1].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax[1].xaxis.get_major_formatter().set_powerlimits((0, 0))
ax[1].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax[1].yaxis.get_major_formatter().set_powerlimits((0, 0))

#ax[1].set_title('')
ax[1].set_xlabel('x [pc]')
ax[1].set_ylabel('z [pc]')

plt.tight_layout()
#plt.savefig('./Plots/.png', bbox_inches='tight')
plt.show()

### 3D arrow plot

from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

y_pos, vy_vel, vz_vel = zip(*sorted(zip(positions[:, 2], velocities[:, 1], velocities[:, 2])))

y_pos = np.array(y_pos)
vy_vel = np.array(vy_vel)
vz_vel = np.array(vz_vel)

vel_data = np.array([vy_vel, vz_vel])
vel_data = vel_data.transpose()

polynomial_features= PolynomialFeatures(degree=7)
y_poly = polynomial_features.fit_transform(y_pos[:, np.newaxis])

model = LinearRegression()
model.fit(y_poly, vel_data)
vy_poly_pred = model.predict(y_poly)

rmse = np.sqrt(mean_squared_error(vel_data,vy_poly_pred)) #root mean square error
r2 = r2_score(vel_data,vy_poly_pred)
print("RMSE:", rmse)
print("R-squared", r2)

fig3d = plt.figure(figsize=(12, 10))
ax3d = fig3d.add_subplot(projection='3d', computed_zorder=False)


#colormap=mpl.cm.Reds
#arrow_map = mpl.cm.ScalarMappable(cmap=colormap, norm=None)
#arrow_map.set_array(angular_momenta[::100, 0])


ax3d.scatter(run_fid_large_mod_data[:,2,-1], run_fid_large_mod_data[:,5,-1], run_fid_large_mod_data[:,6,-1], 
           c='b', alpha=0.1, s=1, zorder=-100)

ax3d.plot(y_pos, vel_data[:, 0], vel_data[:, 1], color='r', alpha=0.5)

#ax3d.quiver(run_fid_large_mod_data[::100,1,-1], run_fid_large_mod_data[::100,2,-1], 
#            run_fid_large_mod_data[::100,3,-1], ang_mom_dir[::100,0], ang_mom_dir[::100,1],
#            ang_mom_dir[::100,2], length=10000, color=colormap(angular_momenta[::100, 0]), zorder=1000) #cmap='spring', angular_momenta[::10, 0]

#plt.colorbar(arrow_map, label='x-dir')
#ax3d.set_xlim(xmin=-0.7e5, xmax=0.8e5)
#ax3d.set_ylim(ymin=-0.5e5, ymax=1.5e5)
#ax3d.set_zlim(zmin=-1.0e5, zmax=1.75e5)
ax3d.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax3d.xaxis.get_major_formatter().set_powerlimits((0, 0))
ax3d.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax3d.yaxis.get_major_formatter().set_powerlimits((0, 0))
ax3d.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax3d.zaxis.get_major_formatter().set_powerlimits((0, 0))

#ax3d.set_title('')
ax3d.set_xlabel('y [pc]', labelpad=15)
ax3d.set_ylabel('vy', labelpad=15)
ax3d.set_zlabel('vz',labelpad=5)

ax3d.view_init(4, 45) #(elevation, rotation)

plt.show()

#%matplotlib inline
fig3d2 = plt.figure(figsize=(12, 12))

# 1:x, 2:y, 3:z, 4:vx, 5:vy, 6:vz
x_label = 'y'
y_label = 'vy'
z_label = 'vz'

x = 2
y = 5
z = 6

angles = [15, 45, 75, 145]
azimuths = [4, 4, 4, 4]

for i in range(4):
    ax3d2 = fig3d2.add_subplot(2, 2, i+1, projection='3d', computed_zorder=False)
    ax3d2.scatter(run_fid_large_mod_data[:,x, -1], run_fid_large_mod_data[:,y, -1], 
                  run_fid_large_mod_data[:, z, -1], c='b', s=1, alpha=0.5)

    ax3d2.set_xlabel(x_label)
    ax3d2.set_ylabel(y_label)
    ax3d2.set_zlabel(z_label)
    ax3d2.set_title(f'Azimuth: {azimuths[i]}, Angle: {angles[i]}')

    ax3d2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax3d2.xaxis.get_major_formatter().set_powerlimits((0, 0))
    ax3d2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax3d2.yaxis.get_major_formatter().set_powerlimits((0, 0))
    ax3d2.zaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax3d2.zaxis.get_major_formatter().set_powerlimits((0, 0))

    ax3d2.view_init(azimuths[i], angles[i])


plt.show()

In [78]:
#run_petar_btest_data, run_petar_btest_header, run_petar_btest_extra, petar_btest_hist_data = extract_data(r'btest2_data', r'Petar_back_test2', 125, 1)

#print(run_petar_btest_data.shape)
#print(run_petar_btest_extra.shape)
#print(run_petar_btest_header.shape)

#petar_btest_halfmass_r = run_petar_btest_extra[0, :]
#print(petar_btest_halfmass_r)

#print(np.max(petar_btest_halfmass_r))
#print(np.min(petar_btest_halfmass_r))

#v_mean_petar_btest, r_mean_petar_btest = means_calc(run_petar_btest_header)

#print(v_mean_petar_btest.shape)
#print(r_mean_petar_btest.shape)

In [77]:
#anim_disc(run_petar_btest_data, run_petar_btest_header, tstep=8, nsteps=125, lims=5.0e4, fsize=(10, 10), 
#          tx=-25000, ty=24000, format_type='gif', fps=6, run='btest2')

In [76]:
#anim_3d(run_petar_btest_data, run_petar_btest_header, tstep=8, nsteps=125, lims=5.0e4, fsize=(10, 10), 
#        marker_size=20, tx=-30500, ty=35500, tz=35500, format_type='gif', fps=6, run='btest2')

In [29]:
#run_at_centre_data, run_at_centre_header, centre_extra_data, centre_hist_data = extract_data(r'mcluster_at_centre_data', r'Trialrun_1000part_at_centre', 30, 1)
#print(run_at_centre_data.shape)
#print(centre_hist_data.shape)
#print(centre_extra_data.shape)

#centre_halfmass_late = centre_extra_data[0, 6]
#centre_halfmass_early = centre_extra_data[0, 1]
#print(centre_halfmass)
#print(centre_hist_data[0, -1, 6])
#print(np.sum(run_at_centre_data[:, 0, 6]))
#print()


#run_above_disc_data, run_above_disc_header, above_extra_data, above_hist_data = extract_data(r'mcluster_above_disc_data', r'Trialrun_1000part_above_disc', 6, 1)
#print(run_above_disc_data.shape)
#print(above_hist_data.shape)
#print(above_extra_data.shape)

#above_halfmass_late = above_extra_data[0, 6]
#above_halfmass_early = above_extra_data[0, 1]
#print(above_halfmass)

In [109]:
#test_fig1, testax1 = plt.subplots(1, 2, figsize=(14, 5))


#testax1[0].minorticks_on()
#testax1[0].plot(centre_hist_data[1, :, 1], centre_hist_data[0, :, 1], color='b')
#testax1.scatter(sorted_data[position[0], 7], cumsum_mass[position[0]], color='r')

#testax1[0].plot(above_hist_data[1, :, 1], above_hist_data[0, :, 1], color='r')
#testax1.scatter(sorted_data2[position2[0], 7], cumsum_mass2[position2[0]], color='g')
#testax1[0].set_xlabel('r')
#testax1[0].set_ylabel('Sum of M')
#testax1[0].set_title('Cumsum hist early')
#testax1[0].axvline(centre_halfmass_early, color='aqua', linestyle='dashed')
#testax1[0].axvline(above_halfmass_early, color='tomato', linestyle='dashed')
#testax1[0].grid(which='both')


#testax1[1].minorticks_on()
#testax1[1].plot(centre_hist_data[1, :, 6], centre_hist_data[0, :, 6], color='b')
#testax1.scatter(sorted_data[position[0], 7], cumsum_mass[position[0]], color='r')

#testax1[1].plot(above_hist_data[1, :, 6], above_hist_data[0, :, 6], color='r')
#testax1.scatter(sorted_data2[position2[0], 7], cumsum_mass2[position2[0]], color='g')
#testax1[1].set_xlabel('r')
#testax1[1].set_ylabel('Sum of M')
#testax1[1].set_title('Cumsum hist late')
#testax1[1].axvline(centre_halfmass_late, color='aqua', linestyle='dashed')
#testax1[1].axvline(above_halfmass_late, color='tomato', linestyle='dashed')
#testax1[1].grid(which='both')

#plt.show()

In [107]:
#my_data, my_header = extract_data(r'mcluster_at_centre_data', r'Trialrun_1000part_at_centre', 30, 1)

#print(np.shape(my_data))
#print(halfmass_radius_cumsum(my_data[:, 0, 0], my_data[:, 7, 0]))
#print(my_extra_data[0, :])
#print(my_extra_data.shape)#

In [105]:
#data_t1 = my_data[:, :, 6]
#print(np.shape(data_t1))
#print(type(data_t1))
#print(data_t1[:, 7])
#print(data_t1[:, 0])
#print()
#print(min(data_t1[:, 7]))
#print(max(data_t1[:, 7]))
#print()


#my_r = my_data[:, 7, 0]

#sorted_data = np.array(sorted(data_t1, key=lambda x:x[7]))
#print(np.shape(sorted_data))
#print(type(sorted_data))
#print(sorted_data[:,7])
#print(sorted_data[:,0])

In [106]:
#cumsum_mass = np.cumsum(sorted_data[:, 0])
#print(cumsum_mass[3200:3300])
#print(len(cumsum_mass))
#print(np.sum(data_t1[:, 0]))
#print(cumsum_mass[-1])
#Mtot = cumsum_mass[-1]
#half_mass = 0.5*Mtot
#print(f'{half_mass = }')

#half_mass_diff = np.abs(cumsum_mass - half_mass)
#closest = np.min(half_mass_diff)
#print(closest)
#position = np.where(half_mass_diff==closest)[0]
#print(half_mass_diff[position])
#print(cumsum_mass[position])
#print(sorted_data[position, 7], position)

In [11]:
#r_halfmass = halfmass_radius_finder(my_data[:, 0, 0], my_data[:, 7, 0])
#print()
#print(r_halfmass)
#print()
#r_halfmass2 = halfmass_radius(my_data[:, 0, 5], my_radii)
#print()
#print(r_halfmass2)

r_test = 6.17909780681924
difference = 808.6573131521109
r_test = 9.268646710228861
difference = 110.85027228494118
r_test = 9.268646710228861
difference = 110.85027228494118

9.268646710228861


# Unused functions

In [ ]:
def halfmass_radius_finder(masses, radii):
    Mtot = np.sum(masses)
    
    r_max = np.max(radii)
    r_test = 0.5*r_max
    r_tests = []
    r_tests.append(r_test)
    n_part_inside = []
    
    masses_inside = masses[radii<=r_test]
    Mtot_inside = np.sum(masses_inside)
    n_part_inside.append(len(masses_inside))
    #print(len(masses_inside))
    difference = 0.5*Mtot-Mtot_inside
    
    if np.abs(difference)<=1e-1:
        return r_test
    
    else:
        if difference<0:
            r_test = r_test - 0.5*r_test
            r_tests.append(r_test)
            
        elif difference>0:
            r_test = r_test + 0.5*r_test
            r_tests.append(r_test)
            
        
        masses_inside = masses[radii<=r_test]
        n_part_inside.append(len(masses_inside))
        #print(len(masses_inside))
        Mtot_inside = np.sum(masses_inside)
        difference = 0.5*Mtot-Mtot_inside
        it = 1
        while np.abs(difference)>1e-1:
            print(f'{r_test = }')
            print(f'{difference = }')
            if difference<0:
                r_test = r_test - 0.5*np.abs(r_test-r_tests[it-1])
                
                r_tests[it] = r_test
                r_tests.append(r_test)
                
                masses_inside = masses[radii<=r_test]
                #print(len(masses_inside))
                n_part_inside.append(len(masses_inside))
                if len(masses_inside)==n_part_inside[it-1]:
                    return r_test
                    break
                Mtot_inside = np.sum(masses_inside)
                difference = 0.5*Mtot-Mtot_inside
                it = it+1
                
                
            elif difference>0:
                r_test = r_test + 0.5*np.abs(r_test-r_tests[it-1])
                
                r_tests[it] = r_test
                r_tests.append(r_test)
                masses_inside = masses[radii<=r_test]
                #print(len(masses_inside))
                n_part_inside.append(len(masses_inside))
                if len(masses_inside)==n_part_inside[it-1]:
                    return r_test
                    break
                Mtot_inside = np.sum(masses_inside)
                difference = 0.5*Mtot-Mtot_inside
                it = it+1
                
        return r_test